In [92]:
import pandas as pd
import numpy as np

df=pd.read_csv('Churn_Modelling.csv')

In [93]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [94]:
X=df.drop(['RowNumber','CustomerId','Surname','Exited'],axis=1)
y=df['Exited']

In [95]:
X['Gender']=pd.get_dummies(X['Gender'],drop_first=True)

In [96]:
X['Geography']=X['Geography'].map(X['Geography'].value_counts().to_dict())

In [97]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,5014,0,42,2,0.00,1,1,1,101348.88
1,608,2477,0,41,1,83807.86,1,0,1,112542.58
2,502,5014,0,42,8,159660.80,3,1,0,113931.57
3,699,5014,0,39,1,0.00,2,0,0,93826.63
4,850,2477,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,5014,1,39,5,0.00,2,1,0,96270.64
9996,516,5014,1,35,10,57369.61,1,1,1,101699.77
9997,709,5014,0,36,7,0.00,1,0,1,42085.58
9998,772,2509,1,42,3,75075.31,2,1,0,92888.52


In [98]:
X.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [101]:
X_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
8371,813,2509,0,29,5,106059.40,1,0,0,187976.88
5027,681,2509,1,37,8,73179.34,2,1,1,25292.53
9234,642,2509,1,49,4,120688.61,1,1,0,24770.22
3944,612,5014,0,31,8,117989.76,1,1,1,54129.86
6862,586,5014,1,33,9,0.00,1,1,0,6975.02
...,...,...,...,...,...,...,...,...,...,...
5734,768,5014,1,54,8,69712.74,1,1,1,69381.05
5191,682,5014,0,58,1,0.00,1,1,1,706.50
5390,735,5014,0,38,1,0.00,3,0,0,92220.12
860,667,5014,1,43,8,190227.46,1,1,0,97508.04


In [102]:
X_train=torch.FloatTensor(X_train.values)
X_test=torch.FloatTensor(X_test.values)

In [103]:
y_train=torch.LongTensor(y_train.values)
y_test=torch.LongTensor(y_test.values)

In [145]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.f1=nn.Linear(10,20)
        self.f2=nn.Linear(20,15)
        self.out=nn.Linear(15,2)
    def forward(self,x):
        x=F.relu(self.f1(x))
        x=F.relu(self.f2(x))
        x=self.out(x)
        return x

In [146]:
'''model=nn.Sequential(nn.Linear(10,20),
                   nn.Linear(20,15),
                   nn.Linear(15,2),
                   nn.Sigmoid())'''

'model=nn.Sequential(nn.Linear(10,20),\n                   nn.Linear(20,15),\n                   nn.Linear(15,2),\n                   nn.Sigmoid())'

In [147]:
model=model()
model.parameters

<bound method Module.parameters of model(
  (f1): Linear(in_features=10, out_features=20, bias=True)
  (f2): Linear(in_features=20, out_features=15, bias=True)
  (out): Linear(in_features=15, out_features=2, bias=True)
)>

In [148]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters())

In [149]:
epochs=50
final_losses=[]
for epoch in range(epochs):
    epoch+=1
    y_pred=model(X_train)
    loss=criterion(y_pred,y_train)
    final_losses.append(loss)
    if epoch%10==1:
        print("epoch number is",epoch," and loss function is ",loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch number is 1  and loss function is  1020.4165649414062
epoch number is 11  and loss function is  255.59738159179688
epoch number is 21  and loss function is  227.6601104736328
epoch number is 31  and loss function is  144.32513427734375
epoch number is 41  and loss function is  47.427547454833984


In [153]:
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        predictions.append(model((data)).argmax().item())
        print(model(data).argmax().item())

0
0
0
0
1
0
1
1
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
1
1
0
0
1
0
0
1
0
0
0
0
0
0
0
1
1
1
1
1
0
0
1
0
0
0
0
0
1
0
1
0
0
1
0
0
1
1
1
0
0
0
0
1
0
0
0
0
1
1
0
0
0
0
1
0
0
0
0
0
0
0
1
1
1
1
1
1
1
0
1
0
0
0
0
1
0
0
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
0
1
0
1
1
0
0
0
0
1
0
0
0
1
1
1
0
0
1
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
1
0
0
1
0
0
1
0
1
1
0
0
0
1
0
1
0
0
0
1
0
1
1
0
1
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
1
1
0
0
0
0
1
1
0
1
0
0
0
0
1
0
0
0
1
0
0
0
0
0
1
1
1
1
1
0
0
0
1
0
0
0
0
1
0
0
1
0
0
0
1
0
1
0
0
1
0
0
1
1
1
0
0
1
0
1
0
1
1
0
0
1
0
0
1
0
0
0
0
1
1
0
0
0
1
0
1
1
0
1
1
0
0
0
1
0
1
1
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
1
0
0
1
1
0
1
1
0
1
1
0
1
1
0
1
0
1
1
0
1
0
1
0
1
0
1
0
1
0
0
0
1
0
0
1
0
1
0
0
1
0
0
0
1
0
1
0
0
1
0
0
0
1
1
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
1
1
0
0
1
0
1
0
1
1
1
0
1
0
0
1
0
0
1
0
1
1
0
0
1
1
1
0
0
0
0
1
0
0
1
0
1
1
1
0
1
1
1
1
0
0
1
0
0
0
1
1
1
0
1
1
0
0
0
1
0
1
1
0
1
0
0
0
1
1
0
0
1
1
1
0
1
1
0
1
1
0
0
0
1
0
0
0
0
1


In [154]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(predictions,y_test)
accuracy

0.5912121212121212

In [155]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predictions,y_test)

array([[1664,  356],
       [ 993,  287]], dtype=int64)